TASK-1 -- SOCIAL MEDIA POST GENERATOR

In [ ]:
%pip install -qU langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 962.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 884.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import userdata

file = open("doc1.txt")
def parse_questions_answers(text):
    lines = text.split('\n')
    qa_dict = {}
    question = None
    answer_lines = []

    for line in lines:
        if line.startswith("Q"):
            if question:
                qa_dict[question] = ' '.join(answer_lines).strip()
            question = line.strip()
            answer_lines = []
        elif line.startswith("Ans."):
            answer_lines.append(line[5:].strip())  # Remove 'Ans. ' prefix
        else:
            answer_lines.append(line.strip())

    # Add the last Q&A pair
    if question:
        qa_dict[question] = ' '.join(answer_lines).strip()

    return qa_dict

# Parse the input text
answers = parse_questions_answers(file.read())

review_template_str = """
    Your job is to generate a social media post
    based on the answers to the questions provided by user.
    Use the following context to generate post. Be as specific
    as possible, but don't make up any information that's not
    provided by user. If you aren't provided the information, just add what
    ever is there in its place. Make the post according to the {tone}
    mentioned by user. Here is a sample post:

    Did you know? {quotes}
    Discover how {case_study}:
    Make bullets of the {achievements}. Also add relevant info from your knowledge.
    {call_to_action} to #{hashtag}

"""

review_template = ChatPromptTemplate.from_template(review_template_str)

quotes = answers.get("Q4. Are there any data, quotes, or testimonials that illustrate the impact?")
case_study = answers.get("Q1. Can you describe the specific case study or transformation story you want to share?")
achievements = answers.get("Q3. What were the key results or outcomes achieved in this story?")
results = answers.get("Q3. What were the key results or outcomes achieved in this story?")
call_to_action = answers.get("Q5. Is there a specific call-to-action?")
hashtag = answers.get("Q6. Are there any specific hashtags you’d like to include?")
tone = answers.get("Q7. Describe your desired tone and style.")

ans = review_template.format(quotes = quotes, case_study = case_study, achievements = achievements, results = results, call_to_action = call_to_action, hashtag = hashtag, tone = tone)
# print(ans)


llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key= userdata.get("OPENAI_API_KEY"))
prompt = f"Modify this {ans} according to the {tone} mentioned by user. Also add supporting facts to this social media post inside the bullets in the post to make it look more appealing. "
response = llm.invoke(prompt)
print(response.content)

Did you know? 75% of classroom-style training is forgotten if it's not implemented within 6 days after. Discover how successful collaboration between Sodexo and CoachHub to accelerate leadership development across the APMEA region.

🌟 The key results achieved in this story include:
- Implementation of scalable coaching programs powered by AI and analytics
- Improved knowledge retention among employees
- Enhanced leadership skills development
- Increased employee engagement across the APMEA region

Join thousands of coachees like Sodexo's Head of FMCG Accounts, Jean Baptiste CALEMARD, on a journey of growth and transformation to explore the greater you. #ExploreTheGreaterYou https://bit.ly/3LMxIHg


TASK-2 -- TIMETABLE GENERATOR

In [41]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd
from google.colab import userdata

# Step 1: Read student data from CSV
data = pd.read_csv("Student Data Rows.csv")

# Assuming 'Name' is the column containing student names -- it should be inside the student data sheet
student_name = input("Enter your name: ")
student = data[data['Name'] == student_name]

# Step 2: Define the review template
review_template_str = """
Your job is to create personalized study plans for students. You have access
to a range of information about the student here {data}. Thoroughly analyze the diverse array of
student-specific data, encompassing their subjects and academic performance across
different subjects, preferred learning styles (visual, auditory, kinesthetic), extracurricular
activities, and distinct personal objectives or challenges (such as preparing for a specific
exam or overcoming a learning difficulty). Synthesize this information to create a study plan that
not only addresses academic requirements but also aligns with each student's unique needs and aspirations.
Ensure that all-inclusive and individualized plans are generated.
Make sure that the study plan caters to each student’s specific needs.
"""

# Step 3: Format the template with student data
review_template = ChatPromptTemplate.from_template(review_template_str)
formatted_template = review_template.format(data=student)

# Step 4: Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=userdata.get("OPENAI_API_KEY"))

# Step 5: Generate personalized study plan
prompt = f"Follow this template:\n\n{formatted_template}\n\nGenerate a personalized study plan for {student_name}."
response = llm.invoke(prompt)
print(response.content)


Enter your name: John Smith
Student: John Smith
Field of Study: Computer Science
Year of Study: 2nd Year
Subjects: Algorithms, Data Structures
Preferred Learning Styles: Visual, Logical
Personal Objectives: Master algorithms for competitive programming.
Challenges: Managing time between studies and part-time job.
Extracurricular Activities: Coding Club, Soccer Team

Based on the information provided, here is a personalized study plan for John Smith:

1. Time Management:
- Create a schedule that allows for dedicated study time for Algorithms and Data Structures while also balancing your part-time job and extracurricular activities.
- Use visual aids such as color-coded calendars or study planners to help organize your time effectively.

2. Visual Learning Techniques:
- Utilize visual learning resources such as diagrams, charts, and videos to enhance your understanding of complex algorithms and data structures.
- Create mind maps or flowcharts to visually represent key concepts and conne